# Ensemble - ALL MODELS

## Environment Setup

### Imports

In [1]:
import os
os.sys.path.append("utils")

import torch
import numpy as np
import pandas as pd
from functools import reduce
import torch.nn.functional as F
from torch.utils.data import DataLoader
from utils.Transforms import Transforms
from monai.transforms import AsDiscrete
from utils.Datasets import EnsembleDataset
from utils.Models import LogisticRegression
from torch.utils.data import SequentialSampler
from utils.Inference import ensemble_inference, model_ensemble_inference

### Config

In [2]:
seed = 33
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

## Load Test Data

In [3]:
subject_ids = pd.read_csv('./data/TEST.csv')['SubjectID'].values

ah_segs, unet_segs, segresnet_segs, untr_segs, gt_segs = [], [], [], [], []
for sid in subject_ids:
    ah_channels, unet_channels, segresnet_channels, untr_channels, gt_channels = [], [], [], [], []
    for channel in ['TC', 'WT', 'ET']:
        ah_channels.append(f'./outputs/AHNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        unet_channels.append(f'./outputs/UNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        segresnet_channels.append(f'./outputs/SegResNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        untr_channels.append(f'./outputs/UNETR/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        gt_channels.append(f'./outputs/gt_segs/test_gt_segs/gt_{sid}_{channel}.npz')
    
    ah_segs.append(ah_channels)
    unet_segs.append(unet_channels)
    segresnet_segs.append(segresnet_channels)
    untr_segs.append(untr_channels)
    gt_segs.append(gt_channels)

# Dataframe
test_df = pd.DataFrame()
test_df['SubjectID'] = subject_ids
test_df['AHNet'] = ah_segs
test_df['UNet'] = unet_segs
test_df['SegResNet'] = segresnet_segs
test_df['UNETR'] = untr_segs
test_df['GT'] = gt_segs

test_df.head()

,SubjectID,AHNet,UNet,SegResNet,UNETR,GT
0,100214B,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100214B_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100214B_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100214B_ET.npz]"
1,100340A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100340A_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100340A_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100340A_ET.npz]"
2,100391A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100391A_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100391A_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100391A_ET.npz]"
3,100190B,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100190B_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100190B_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100190B_ET.npz]"
4,100142A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_1

## Ensemble Functions

In [4]:
# Average
def avg(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.mean(stacked_predictions, dim=0)

#---------------------------------#

# Weighted Average
def weigthed_avg(predictions):

    # Weights
    weights = np.ones(len(predictions))
    if len(predictions) == 4: # All models
        weights = np.array([0.546402, 0.648790, 0.428639, 0.547957])
    elif len(predictions) == 3: # 3 models
        weights = np.array([0.546402, 0.648790, 0.547957])
    elif len(predictions) == 2: # 2 models
        weights = np.array([0.546402, 0.547957])
    weights = weights / np.sum(weights)
    weights = torch.tensor(weights).float()
    
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.sum(stacked_predictions * weights[:, None, None, None, None, None], dim=0)

#---------------------------------#

# Median
def median(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.median(stacked_predictions, dim=0).values

#---------------------------------#

# Maximun Confidence
def max_confidence(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.max(stacked_predictions, dim=0).values

#---------------------------------#

# Majority Voting
def majority_voting(predictions):
    transforms =[AsDiscrete(threshold=0.3), AsDiscrete(threshold=0.3), AsDiscrete(threshold=0.3), AsDiscrete(threshold=0.3)]
    predictions = [transform(prediction) for transform, prediction in zip(transforms, predictions)]
    stacked_predictions = torch.stack(predictions, dim=0)  
    return torch.mode(stacked_predictions, dim=0).values

In [5]:
# Dempter-Shafer
def dempster_rule(mass1, mass2):

    K = 1 / (1 - mass1[..., 0] * mass2[..., 1] - mass1[..., 1] * mass2[..., 0])

    combined_mass = torch.zeros_like(mass1)
    combined_mass[..., 0] = K * (mass1[..., 0] * mass2[..., 0] + mass1[..., 0] * mass2[..., 2] + mass1[..., 2] * mass2[..., 0])
    combined_mass[..., 1] = K * (mass1[..., 1] * mass2[..., 1] + mass1[..., 1] * mass2[..., 2] + mass1[..., 2] * mass2[..., 1])
    combined_mass[..., 2] = K * (mass1[..., 0] * mass2[..., 1] + mass1[..., 1] * mass2[..., 0] + mass1[..., 2] * mass2[..., 2])
    
    return combined_mass

def dempster_shafer_inference(predictions):

    # Params
    predictions = [pred.squeeze(0) for pred in predictions]

    # Mass Beliefs for {tumor, non-tumor} - (3, 3, x, y, z) - (masses, channels, x, y, z)
    masses = []	
    for pred in predictions:
        epsilon = 1 - torch.max(pred) 
        tumor_mass = pred
        non_tumor_mass = 1 - pred
        uncertainty_mass = torch.full_like(pred, epsilon)
        masses.append(torch.stack([tumor_mass, non_tumor_mass, uncertainty_mass], dim=-1))

    combined_mass = reduce(dempster_rule, masses)

    # Normalized Mass
    combined_mass_sum = combined_mass.sum(dim=-1, keepdim=True)
    normalized_mass = combined_mass / combined_mass_sum
    
    return normalized_mass[..., 0].unsqueeze(0)   

In [16]:
# CRF - Mean Field Approximation
def crf_mean(predictions, sigma_spatial=3.0, sigma_bilateral=1.0, iterations=10, alpha=0.7):
    # Weighted Average
    avg_pred = weigthed_avg(predictions)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    avg_pred = avg_pred.to(device)
    Q = avg_pred.clone()

    for _ in range(iterations):
        # Spatial
        Q_spatial = F.avg_pool3d(Q, kernel_size=3, stride=1, padding=1)

        # Bilateral
        diff = (Q.unsqueeze(2) - Q.unsqueeze(1)).pow(2)
        bilateral_weight = torch.exp(-diff / (2 * sigma_bilateral**2))
        Q_bilateral = (Q.unsqueeze(2) * bilateral_weight).sum(1) / bilateral_weight.sum(1)

        # Combine spatial and bilateral
        Q_new = Q * torch.exp((Q_spatial - Q) / sigma_spatial + (Q_bilateral - Q) / sigma_bilateral)
        Q_new = Q_new / Q_new.sum(dim=1, keepdim=True)

        Q = Q_new
        
    final_pred = alpha * avg_pred + (1 - alpha) * Q

    return final_pred.cpu()

In [17]:
# CRF - Energy Minimization
def crf_energy(predictions, iterations=5, sigma_spatial=3.0, sigma_bilateral=1.0, learning_rate=1e-3, alpha=0.7):
    # Weighted Average
    avg_pred = weigthed_avg(predictions)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    avg_pred = avg_pred.to(device)

    Q = avg_pred.clone().requires_grad_(True)

    for _ in range(iterations):
        # Compute energy
        unary_energy = -torch.sum(avg_pred * torch.log(Q + 1e-10))
        
        # Spatial pairwise energy
        Q_spatial = F.avg_pool3d(Q, kernel_size=3, stride=1, padding=1)
        spatial_energy = torch.sum((Q - Q_spatial).pow(2)) / (2 * sigma_spatial**2)
        
        # Bilateral pairwise energy
        diff = (Q.unsqueeze(2) - Q.unsqueeze(1)).pow(2)
        bilateral_weight = torch.exp(-diff / (2 * sigma_bilateral**2))
        Q_bilateral = (Q.unsqueeze(2) * bilateral_weight).sum(1) / bilateral_weight.sum(1)
        bilateral_energy = torch.sum((Q - Q_bilateral).pow(2)) / (2 * sigma_bilateral**2)
        
        # Total energy
        energy = unary_energy + spatial_energy + bilateral_energy
        
        # Update Q
        energy.backward()
        with torch.no_grad():
            Q -= learning_rate * Q.grad
            Q.grad.zero_()
        Q = F.softmax(Q, dim=1)
        Q = Q.detach().requires_grad_(True)

    final_pred = alpha * avg_pred + (1 - alpha) * Q
    
    return final_pred.detach().cpu()

## Ensembles - All Models

In [8]:
path = './outputs/Ensemble/'
models = ['AHNet', 'SegResNet', 'UNet', 'UNETR']

In [9]:
# Average
df = ensemble_inference(models, test_df, avg, threshold=0.4)
df.to_csv(f'{path}ensemble_avg.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.630636,0.662461,0.673749,0.533071,6.677419,6.516129,87.129032,8.709677,8.709677,5.129032,16592.322581,16591.774194,14575.709677,17105.387097,17985.870968,13810.806452
std,0.259768,0.255310,0.260801,0.293511,5.906985,5.977017,166.985177,9.103822,9.103822,4.951376,24168.923954,24207.681579,22133.776319,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,7.000000,6.000000,0.000000,44.000000,44.000000,0.000000
25%,0.484863,0.554968,0.560128,0.282297,3.000000,2.500000,5.500000,2.000000,2.000000,1.500000,1057.000000,1048.000000,662.500000,802.000000,802.000000,410.000000
50%,0.676840,0.717632,0.698113,0.542388,5.000000,5.000000,14.000000,6.000000,6.000000,4.000000,4993.000000,4986.000000,3298.000000,3847.000000,4138.000000,1944.000000
75%,0.842653,0.869613,0.885937,0.769723,8.000000,8.000000,65.000000,13.500000,13.500000,7.000000,29342.500000,29308.000000,27872.000000,26305.000000,27036.500000,23520.500000
max,0.946255,0.955846,0.958104,0.924816,25.000000,25.000000,684.000000,43.000000,43.000000,21.000000,90746.000000,90706.000000,83437.000000,102770.000000,105729.000000,92221.000000


In [10]:
# Weighted Average
df = ensemble_inference(models, test_df, weigthed_avg, threshold=0.4)
df.to_csv(f'{path}/ensemble_weighted_avg.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.635900,0.667215,0.679828,0.538870,6.903226,6.419355,109.774194,8.709677,8.709677,5.129032,16629.322581,16633.903226,14432.903226,17105.387097,17985.870968,13810.806452
std,0.257415,0.252058,0.256418,0.295345,5.985287,5.863868,204.315884,9.103822,9.103822,4.951376,24313.317312,24377.260244,22128.085428,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,6.000000,5.000000,0.000000,44.000000,44.000000,0.000000
25%,0.498647,0.566927,0.570827,0.290649,2.500000,2.000000,6.000000,2.000000,2.000000,1.500000,1042.000000,1032.500000,635.000000,802.000000,802.000000,410.000000
50%,0.675439,0.723906,0.727524,0.549588,5.000000,5.000000,21.000000,6.000000,6.000000,4.000000,4940.000000,4945.000000,3194.000000,3847.000000,4138.000000,1944.000000
75%,0.846335,0.871845,0.890754,0.776012,8.500000,8.000000,103.500000,13.500000,13.500000,7.000000,29476.500000,29444.500000,27817.000000,26305.000000,27036.500000,23520.500000
max,0.947493,0.956289,0.958427,0.927762,25.000000,25.000000,922.000000,43.000000,43.000000,21.000000,91540.000000,91606.000000,83508.000000,102770.000000,105729.000000,92221.000000


In [11]:
# Median
df = ensemble_inference(models, test_df, median, threshold=0.4)
df.to_csv(f'{path}/ensemble_median.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.592251,0.625845,0.626673,0.516698,5.903226,5.838710,8.161290,8.709677,8.709677,5.129032,14633.161290,14741.741935,12157.387097,17105.387097,17985.870968,13810.806452
std,0.286974,0.276813,0.291834,0.320492,5.752419,6.061335,9.612827,9.103822,9.103822,4.951376,22319.908553,22558.457920,19780.442551,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.398890,0.471294,0.466704,0.192902,2.000000,2.000000,2.500000,2.000000,2.000000,1.500000,732.000000,718.500000,465.500000,802.000000,802.000000,410.000000
50%,0.653768,0.684043,0.717003,0.593920,4.000000,4.000000,5.000000,6.000000,6.000000,4.000000,3549.000000,3546.000000,2550.000000,3847.000000,4138.000000,1944.000000
75%,0.834203,0.867059,0.874760,0.793132,7.500000,7.000000,10.000000,13.500000,13.500000,7.000000,24729.000000,24901.500000,22015.000000,26305.000000,27036.500000,23520.500000
max,0.951454,0.957341,0.959509,0.937512,22.000000,22.000000,39.000000,43.000000,43.000000,21.000000,83221.000000,83512.000000,74275.000000,102770.000000,105729.000000,92221.000000


In [12]:
# Maximum Confidence
df = ensemble_inference(models, test_df, max_confidence, threshold=0.4)
df.to_csv(f'{path}ensemble_max_confidence.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.519687,0.557690,0.569230,0.431639,34.161290,31.677419,34.580645,8.709677,8.709677,5.129032,21617.032258,21632.225806,21275.935484,17105.387097,17985.870968,13810.806452
std,0.312949,0.318218,0.327271,0.302892,54.264228,48.573235,58.826170,9.103822,9.103822,4.951376,27887.914179,27994.480121,27943.434225,25565.209783,26912.616906,22937.645993
min,0.007680,0.012817,0.010222,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,0.000000,1986.000000,2005.000000,1813.000000,44.000000,44.000000,0.000000
25%,0.214851,0.249721,0.248241,0.130362,11.500000,10.500000,8.000000,2.000000,2.000000,1.500000,4126.500000,4106.500000,3889.500000,802.000000,802.000000,410.000000
50%,0.638677,0.676597,0.697987,0.496722,21.000000,19.000000,15.000000,6.000000,6.000000,4.000000,7842.000000,7704.000000,6868.000000,3847.000000,4138.000000,1944.000000
75%,0.783556,0.841341,0.858065,0.674008,33.500000,30.500000,25.000000,13.500000,13.500000,7.000000,35506.500000,35595.000000,35823.500000,26305.000000,27036.500000,23520.500000
max,0.916666,0.936059,0.937123,0.876816,263.000000,244.000000,284.000000,43.000000,43.000000,21.000000,107651.000000,107975.000000,106480.000000,102770.000000,105729.000000,92221.000000


In [13]:
# Majority Voting
majority_vote_df = ensemble_inference(models, test_df, majority_voting)
majority_vote_df.to_csv(f'{path}ensemble_majority_vote.csv', index=False)
majority_vote_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.602996,0.633261,0.638629,0.525343,6.225806,5.709677,8.129032,8.709677,8.709677,5.129032,15176.096774,15288.451613,12763.709677,17105.387097,17985.870968,13810.806452
std,0.280308,0.273121,0.282172,0.314649,6.540691,5.734071,10.111848,9.103822,9.103822,4.951376,23072.307900,23298.294801,20540.390660,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.419860,0.491230,0.492867,0.213326,2.000000,2.000000,2.000000,2.000000,2.000000,1.500000,806.000000,809.500000,527.000000,802.000000,802.000000,410.000000
50%,0.646907,0.677307,0.725038,0.587416,4.000000,4.000000,5.000000,6.000000,6.000000,4.000000,3673.000000,3696.000000,2722.000000,3847.000000,4138.000000,1944.000000
75%,0.842977,0.869454,0.879870,0.789589,8.000000,7.000000,8.000000,13.500000,13.500000,7.000000,25870.500000,26098.000000,23754.500000,26305.000000,27036.500000,23520.500000
max,0.950614,0.957061,0.959217,0.936127,29.000000,25.000000,43.000000,43.000000,43.000000,21.000000,86124.000000,86616.000000,77293.000000,102770.000000,105729.000000,92221.000000


In [14]:
# Dempter-Shafer
ds_df = ensemble_inference(models, test_df, dempster_shafer_inference, threshold=0.1)
ds_df.to_csv(f'{path}ensemble_dempster_shafer.csv', index=False)
ds_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.577118,0.606080,0.618176,0.490236,6.290323,6.225806,6.741935,8.709677,8.709677,5.129032,15446.258065,15558.225806,13528.290323,17105.387097,17985.870968,13810.806452
std,0.279771,0.276253,0.281809,0.313583,5.768845,5.649246,6.093536,9.103822,9.103822,4.951376,23810.414574,23963.213483,21562.279380,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,16.000000,12.000000,0.000000,44.000000,44.000000,0.000000
25%,0.442785,0.495962,0.520909,0.213849,4.000000,4.000000,3.000000,2.000000,2.000000,1.500000,698.000000,724.000000,533.500000,802.000000,802.000000,410.000000
50%,0.565420,0.608258,0.612368,0.492505,5.000000,5.000000,5.000000,6.000000,6.000000,4.000000,3781.000000,3802.000000,3008.000000,3847.000000,4138.000000,1944.000000
75%,0.834014,0.862115,0.876348,0.775048,7.000000,7.000000,8.500000,13.500000,13.500000,7.000000,27914.500000,28144.500000,26598.000000,26305.000000,27036.500000,23520.500000
max,0.948727,0.954471,0.957102,0.934608,28.000000,28.000000,27.000000,43.000000,43.000000,21.000000,89020.000000,89231.000000,81336.000000,102770.000000,105729.000000,92221.000000


In [15]:
# CRF - Mean Field Approximation
df = ensemble_inference(models, test_df, crf_mean, threshold=0.4)
df.to_csv(f'{path}ensemble_crf_mean.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.635137,0.666215,0.678172,0.538820,6.645161,6.548387,123.709677,8.709677,8.709677,5.129032,16324.032258,16321.354839,14036.870968,17105.387097,17985.870968,13810.806452
std,0.259541,0.253901,0.258408,0.298055,5.689454,6.048905,232.605129,9.103822,9.103822,4.951376,23963.957376,24025.909997,21674.041599,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,5.000000,2.000000,0.000000,44.000000,44.000000,0.000000
25%,0.500498,0.559208,0.571193,0.293311,3.500000,2.000000,7.000000,2.000000,2.000000,1.500000,992.000000,974.000000,584.000000,802.000000,802.000000,410.000000
50%,0.688387,0.722104,0.740951,0.561484,5.000000,5.000000,26.000000,6.000000,6.000000,4.000000,4791.000000,4748.000000,3074.000000,3847.000000,4138.000000,1944.000000
75%,0.846332,0.870643,0.889466,0.781494,8.500000,8.000000,126.500000,13.500000,13.500000,7.000000,29037.000000,29010.000000,27191.000000,26305.000000,27036.500000,23520.500000
max,0.948949,0.957131,0.959748,0.929968,25.000000,25.000000,1103.000000,43.000000,43.000000,21.000000,90066.000000,90116.000000,81821.000000,102770.000000,105729.000000,92221.000000


In [16]:
# CRF - Energy Minimization
df = ensemble_inference(models, test_df, crf_energy, threshold=0.4)
df.to_csv(f'{path}ensemble_crf_energy.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.635099,0.666059,0.678025,0.538904,6.612903,6.387097,123.451613,8.709677,8.709677,5.129032,16321.967742,16317.000000,14038.354839,17105.387097,17985.870968,13810.806452
std,0.259750,0.253963,0.259043,0.297947,5.643152,6.059029,231.764512,9.103822,9.103822,4.951376,23962.280433,24021.201451,21676.087175,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,5.000000,2.000000,0.000000,44.000000,44.000000,0.000000
25%,0.500684,0.558539,0.571357,0.293646,3.500000,2.000000,7.000000,2.000000,2.000000,1.500000,992.500000,973.500000,583.500000,802.000000,802.000000,410.000000
50%,0.691589,0.722104,0.740583,0.561484,5.000000,5.000000,26.000000,6.000000,6.000000,4.000000,4791.000000,4746.000000,3075.000000,3847.000000,4138.000000,1944.000000
75%,0.846302,0.870695,0.889422,0.781489,8.000000,8.000000,125.500000,13.500000,13.500000,7.000000,29038.500000,29009.000000,27190.500000,26305.000000,27036.500000,23520.500000
max,0.948939,0.957131,0.959761,0.929925,25.000000,25.000000,1097.000000,43.000000,43.000000,21.000000,90051.000000,90081.000000,81829.000000,102770.000000,105729.000000,92221.000000


In [17]:
# Data
transforms = Transforms(seed=33)
test_dataset = EnsembleDataset(test_df.drop(columns = ['SubjectID']), transform=transforms.val_ensemble(), size = None)
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle = False, sampler = test_sampler)
subjects = pd.read_csv('./data/TEST.csv')['SubjectID'].values

# Model
model = LogisticRegression(12, 3)
model.load_state_dict(torch.load(f'{path}LogRegCheckpoints/LogisticRegression_2.pt'))

#---------------------------------------------------------#

# Logistic Regression - No TTA
df = model_ensemble_inference(subjects, test_loader, model, spatial_size=12, threshold=0.9)
df.to_csv(f'{path}ensemble_logreg.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.591372,0.633501,0.645053,0.488979,6.451613,5.967742,8.129032,8.709677,8.709677,5.129032,16603.354839,15922.129032,16926.645161,17105.387097,17985.870968,13810.806452
std,0.269072,0.275739,0.281359,0.275928,6.152716,6.590315,7.710348,9.103822,9.103822,4.951376,24234.257254,23657.322502,24612.208108,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,3.000000,44.000000,44.000000,0.000000
25%,0.424680,0.518179,0.513945,0.288068,2.000000,2.000000,2.500000,2.000000,2.000000,1.500000,979.000000,796.000000,1090.500000,802.000000,802.000000,410.000000
50%,0.628824,0.682927,0.715011,0.501049,4.000000,4.000000,5.000000,6.000000,6.000000,4.000000,4922.000000,4432.000000,5163.000000,3847.000000,4138.000000,1944.000000
75%,0.813610,0.869493,0.883609,0.702449,8.000000,7.000000,12.000000,13.500000,13.500000,7.000000,29586.500000,28904.500000,29824.000000,26305.000000,27036.500000,23520.500000
max,0.938326,0.955768,0.959606,0.900631,27.000000,28.000000,29.000000,43.000000,43.000000,21.000000,90315.000000,87750.000000,91928.000000,102770.000000,105729.000000,92221.000000


## Ensemble - 3 Models - AHNet, SegResNet & UNETR

In [8]:
path = './outputs/Ensemble_3/'
models = ['AHNet', 'SegResNet', 'UNETR']

In [9]:
# Average
df = ensemble_inference(models, test_df, avg, threshold=0.4)
df.to_csv(f'{path}ensemble_avg.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.642900,0.675335,0.684806,0.547999,7.870968,7.032258,16.096774,8.709677,8.709677,5.129032,16631.483871,16666.580645,13891.870968,17105.387097,17985.870968,13810.806452
std,0.252928,0.241508,0.246799,0.303004,7.830462,6.374344,21.357832,9.103822,9.103822,4.951376,25129.788233,25262.459670,22386.062110,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.523253,0.570072,0.575907,0.320578,3.000000,3.000000,3.000000,2.000000,2.000000,1.500000,982.500000,960.000000,551.500000,802.000000,802.000000,410.000000
50%,0.637359,0.703949,0.753131,0.577692,5.000000,5.000000,8.000000,6.000000,6.000000,4.000000,4718.000000,4724.000000,2852.000000,3847.000000,4138.000000,1944.000000
75%,0.855782,0.873027,0.898497,0.793281,9.500000,9.500000,21.500000,13.500000,13.500000,7.000000,30380.000000,30354.000000,28023.000000,26305.000000,27036.500000,23520.500000
max,0.949085,0.955557,0.958394,0.935284,34.000000,26.000000,99.000000,43.000000,43.000000,21.000000,95617.000000,95689.000000,84916.000000,102770.000000,105729.000000,92221.000000


In [10]:
# Weighted Average
df = ensemble_inference(models, test_df, weigthed_avg, threshold=0.4)
df.to_csv(f'{path}/ensemble_weighted_avg.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.645398,0.676805,0.687453,0.552012,7.193548,6.967742,14.000000,8.709677,8.709677,5.129032,16555.032258,16586.225806,13788.322581,17105.387097,17985.870968,13810.806452
std,0.251027,0.240210,0.243862,0.301197,6.935029,6.635681,17.189144,9.103822,9.103822,4.951376,24977.822062,25104.340697,22205.619714,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.534398,0.576200,0.580152,0.331640,3.500000,3.000000,3.000000,2.000000,2.000000,1.500000,995.500000,970.000000,566.000000,802.000000,802.000000,410.000000
50%,0.631351,0.713736,0.762651,0.576646,5.000000,5.000000,7.000000,6.000000,6.000000,4.000000,4724.000000,4729.000000,2863.000000,3847.000000,4138.000000,1944.000000
75%,0.856444,0.873076,0.898498,0.794745,8.500000,8.500000,19.000000,13.500000,13.500000,7.000000,29923.500000,29944.500000,27641.000000,26305.000000,27036.500000,23520.500000
max,0.949815,0.956011,0.959204,0.935853,30.000000,27.000000,71.000000,43.000000,43.000000,21.000000,95005.000000,95181.000000,84273.000000,102770.000000,105729.000000,92221.000000


In [11]:
# Median
df = ensemble_inference(models, test_df, median, threshold=0.4)
df.to_csv(f'{path}/ensemble_median.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.637875,0.672492,0.680239,0.541232,8.709677,7.967742,9.612903,8.709677,8.709677,5.129032,16071.645161,16227.548387,13444.032258,17105.387097,17985.870968,13810.806452
std,0.254386,0.241583,0.246623,0.308399,7.751101,7.467636,11.238853,9.103822,9.103822,4.951376,24189.381509,24527.321182,21631.560320,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.529374,0.570772,0.579417,0.291528,3.500000,3.500000,2.500000,2.000000,2.000000,1.500000,974.000000,953.000000,556.000000,802.000000,802.000000,410.000000
50%,0.613806,0.693872,0.745872,0.567229,5.000000,5.000000,6.000000,6.000000,6.000000,4.000000,4626.000000,4656.000000,2851.000000,3847.000000,4138.000000,1944.000000
75%,0.853084,0.873338,0.894805,0.791979,12.500000,9.000000,12.000000,13.500000,13.500000,7.000000,28907.500000,29085.500000,26872.000000,26305.000000,27036.500000,23520.500000
max,0.949471,0.956106,0.957815,0.936082,30.000000,28.000000,47.000000,43.000000,43.000000,21.000000,92858.000000,93420.000000,82590.000000,102770.000000,105729.000000,92221.000000


In [12]:
# Maximum Confidence
df = ensemble_inference(models, test_df, max_confidence, threshold=0.4)
df.to_csv(f'{path}ensemble_max_confidence.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.610785,0.649291,0.657078,0.522498,20.032258,18.935484,84.741935,8.709677,8.709677,5.129032,20064.064516,20117.387097,17951.483871,17105.387097,17985.870968,13810.806452
std,0.251513,0.237557,0.251231,0.283070,11.870085,12.179041,145.158986,9.103822,9.103822,4.951376,28396.293512,28495.471263,26651.570137,25565.209783,26912.616906,22937.645993
min,0.108131,0.120713,0.095548,0.000000,4.000000,3.000000,0.000000,1.000000,1.000000,0.000000,117.000000,88.000000,0.000000,44.000000,44.000000,0.000000
25%,0.466810,0.519662,0.517151,0.288739,10.500000,9.000000,11.500000,2.000000,2.000000,1.500000,1402.500000,1414.500000,1001.000000,802.000000,802.000000,410.000000
50%,0.668579,0.704439,0.765412,0.593307,18.000000,14.000000,23.000000,6.000000,6.000000,4.000000,6221.000000,6254.000000,4554.000000,3847.000000,4138.000000,1944.000000
75%,0.828359,0.850638,0.868865,0.759921,26.500000,28.500000,79.500000,13.500000,13.500000,7.000000,35372.500000,35460.500000,33510.500000,26305.000000,27036.500000,23520.500000
max,0.924435,0.937159,0.938148,0.897998,50.000000,53.000000,584.000000,43.000000,43.000000,21.000000,107428.000000,107767.000000,100362.000000,102770.000000,105729.000000,92221.000000


In [13]:
# Majority Voting
majority_vote_df = ensemble_inference(models, test_df, majority_voting)
majority_vote_df.to_csv(f'{path}ensemble_majority_vote.csv', index=False)
majority_vote_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.641560,0.674658,0.684467,0.546810,8.806452,8.225806,9.387097,8.709677,8.709677,5.129032,16842.516129,17002.580645,14252.354839,17105.387097,17985.870968,13810.806452
std,0.248368,0.236819,0.240307,0.300191,7.717596,7.419388,11.062481,9.103822,9.103822,4.951376,25211.120421,25523.302593,22620.338032,25565.209783,26912.616906,22937.645993
min,0.003810,0.000000,0.011429,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,44.000000,44.000000,0.000000
25%,0.524193,0.564680,0.565742,0.332082,3.500000,3.000000,2.000000,2.000000,2.000000,1.500000,1077.500000,1073.500000,637.000000,802.000000,802.000000,410.000000
50%,0.624035,0.710088,0.759918,0.548019,6.000000,5.000000,5.000000,6.000000,6.000000,4.000000,4886.000000,4910.000000,3168.000000,3847.000000,4138.000000,1944.000000
75%,0.853133,0.874410,0.892536,0.788852,10.000000,11.500000,11.500000,13.500000,13.500000,7.000000,30742.000000,30948.500000,28709.000000,26305.000000,27036.500000,23520.500000
max,0.944800,0.952258,0.954205,0.931873,30.000000,29.000000,45.000000,43.000000,43.000000,21.000000,96593.000000,97200.000000,86239.000000,102770.000000,105729.000000,92221.000000


In [14]:
# Dempter-Shafer
ds_df = ensemble_inference(models, test_df, dempster_shafer_inference, threshold=0.1)
ds_df.to_csv(f'{path}ensemble_dempster_shafer.csv', index=False)
ds_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.601745,0.633941,0.644781,0.505325,8.516129,8.419355,9.290323,8.709677,8.709677,5.129032,16802.903226,16908.709677,14467.838710,17105.387097,17985.870968,13810.806452
std,0.264539,0.254862,0.260568,0.308289,7.719546,7.873052,10.687044,9.103822,9.103822,4.951376,25970.063724,26162.844280,23447.012299,25565.209783,26912.616906,22937.645993
min,0.011385,0.022727,0.011429,0.000000,2.000000,1.000000,0.000000,1.000000,1.000000,0.000000,2.000000,1.000000,0.000000,44.000000,44.000000,0.000000
25%,0.456363,0.526884,0.549712,0.230807,3.500000,4.000000,2.500000,2.000000,2.000000,1.500000,850.000000,850.000000,490.000000,802.000000,802.000000,410.000000
50%,0.585188,0.640000,0.629866,0.503819,6.000000,6.000000,6.000000,6.000000,6.000000,4.000000,4183.000000,4317.000000,3003.000000,3847.000000,4138.000000,1944.000000
75%,0.849310,0.874172,0.884706,0.785920,11.000000,10.500000,10.000000,13.500000,13.500000,7.000000,31359.500000,31485.000000,29419.000000,26305.000000,27036.500000,23520.500000
max,0.937775,0.944772,0.947060,0.921493,31.000000,32.000000,47.000000,43.000000,43.000000,21.000000,98062.000000,98116.000000,88349.000000,102770.000000,105729.000000,92221.000000


In [18]:
# CRF - Mean Field Approximation
df = ensemble_inference(models, test_df, crf_mean, threshold=0.4)
df.to_csv(f'{path}ensemble_crf_mean.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.643267,0.676059,0.684608,0.548649,7.193548,6.645161,11.483871,8.709677,8.709677,5.129032,16231.225806,16279.806452,13430.064516,17105.387097,17985.870968,13810.806452
std,0.253890,0.242339,0.247343,0.304838,6.920594,6.457803,13.220870,9.103822,9.103822,4.951376,24534.594601,24696.136302,21763.150106,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.533568,0.578342,0.581405,0.323960,3.500000,3.000000,3.000000,2.000000,2.000000,1.500000,965.000000,935.000000,529.500000,802.000000,802.000000,410.000000
50%,0.635036,0.703195,0.752616,0.582860,5.000000,4.000000,6.000000,6.000000,6.000000,4.000000,4661.000000,4643.000000,2750.000000,3847.000000,4138.000000,1944.000000
75%,0.855568,0.871944,0.898139,0.796715,8.500000,8.500000,16.500000,13.500000,13.500000,7.000000,28962.000000,28988.000000,26655.000000,26305.000000,27036.500000,23520.500000
max,0.951052,0.956996,0.959822,0.937410,30.000000,26.000000,51.000000,43.000000,43.000000,21.000000,93512.000000,93718.000000,82698.000000,102770.000000,105729.000000,92221.000000


In [19]:
# CRF - Energy Minimization
df = ensemble_inference(models, test_df, crf_energy, threshold=0.4)
df.to_csv(f'{path}ensemble_crf_energy.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.642601,0.675014,0.683434,0.548793,7.580645,6.903226,11.806452,8.709677,8.709677,5.129032,16208.903226,16254.000000,13433.935484,17105.387097,17985.870968,13810.806452
std,0.254197,0.242991,0.247759,0.304729,7.297370,6.410173,13.731762,9.103822,9.103822,4.951376,24532.480847,24693.675019,21767.654772,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.533431,0.577694,0.581337,0.323906,2.500000,3.000000,3.000000,2.000000,2.000000,1.500000,961.500000,934.500000,530.500000,802.000000,802.000000,410.000000
50%,0.637398,0.703398,0.752826,0.582716,5.000000,5.000000,6.000000,6.000000,6.000000,4.000000,4655.000000,4638.000000,2751.000000,3847.000000,4138.000000,1944.000000
75%,0.855494,0.872075,0.898147,0.796974,9.000000,9.000000,17.000000,13.500000,13.500000,7.000000,28955.500000,28980.500000,26658.000000,26305.000000,27036.500000,23520.500000
max,0.951048,0.956983,0.959822,0.937292,30.000000,25.000000,53.000000,43.000000,43.000000,21.000000,93493.000000,93684.000000,82721.000000,102770.000000,105729.000000,92221.000000


In [20]:
# Data
transforms = Transforms(seed=33)
test_dataset = EnsembleDataset(test_df.drop(columns = ['SubjectID']), transform=transforms.val_ensemble(), size = None, include_models=[True, True, False, True])
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle = False, sampler = test_sampler)
subjects = pd.read_csv('./data/TEST.csv')['SubjectID'].values

# Model
model = LogisticRegression(9, 3)
model.load_state_dict(torch.load(f'{path}LogRegCheckpoints/LogisticRegression_2.pt'))

#---------------------------------------------------------#

# Logistic Regression - No TTA
df = model_ensemble_inference(subjects, test_loader, model, spatial_size=9, threshold=0.9)
df.to_csv(f'{path}ensemble_logreg.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.612213,0.663321,0.669514,0.489864,8.774194,9.516129,9.838710,8.709677,8.709677,5.129032,18290.258065,19003.064516,18774.000000,17105.387097,17985.870968,13810.806452
std,0.248036,0.251599,0.256710,0.271166,8.208571,8.286016,8.426533,9.103822,9.103822,4.951376,27038.498033,27869.187866,27409.691265,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,10.000000,44.000000,44.000000,0.000000
25%,0.462423,0.554807,0.542016,0.302372,3.500000,4.000000,5.000000,2.000000,2.000000,1.500000,1056.000000,1168.000000,1187.500000,802.000000,802.000000,410.000000
50%,0.654818,0.729425,0.753564,0.517980,6.000000,7.000000,8.000000,6.000000,6.000000,4.000000,4991.000000,5138.000000,5175.000000,3847.000000,4138.000000,1944.000000
75%,0.818042,0.879390,0.884497,0.689917,9.000000,10.000000,12.000000,13.500000,13.500000,7.000000,33123.500000,34406.500000,33904.500000,26305.000000,27036.500000,23520.500000
max,0.926134,0.946958,0.942607,0.888838,31.000000,33.000000,36.000000,43.000000,43.000000,21.000000,102134.000000,105173.000000,103049.000000,102770.000000,105729.000000,92221.000000


## Ensemble - 2 Models - AHNet & UNETR

In [26]:
path = './outputs/Ensemble_2/'
models = ['AHNet', 'UNETR']

In [27]:
# Average
df = ensemble_inference(models, test_df, avg, threshold=0.4)
df.to_csv(f'{path}ensemble_avg.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.605769,0.643667,0.650360,0.508975,13.677419,10.612903,150.774194,8.709677,8.709677,5.129032,17663.193548,17646.387097,15357.774194,17105.387097,17985.870968,13810.806452
std,0.265101,0.256697,0.265615,0.306157,11.341332,9.376131,243.939160,9.103822,9.103822,4.951376,26408.863503,26472.330780,24240.955873,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.448500,0.509076,0.484161,0.232632,6.000000,4.000000,15.500000,2.000000,2.000000,1.500000,880.000000,851.500000,517.000000,802.000000,802.000000,410.000000
50%,0.630689,0.637417,0.718654,0.551874,11.000000,8.000000,49.000000,6.000000,6.000000,4.000000,4740.000000,4738.000000,2982.000000,3847.000000,4138.000000,1944.000000
75%,0.843353,0.869894,0.893236,0.770017,18.000000,13.500000,153.500000,13.500000,13.500000,7.000000,32329.000000,32220.000000,30417.000000,26305.000000,27036.500000,23520.500000
max,0.939770,0.948976,0.951038,0.919298,44.000000,35.000000,1049.000000,43.000000,43.000000,21.000000,100554.000000,100044.000000,91481.000000,102770.000000,105729.000000,92221.000000


In [28]:
# Weighted Average
df = ensemble_inference(models, test_df, weigthed_avg, threshold=0.4)
df.to_csv(f'{path}/ensemble_weighted_avg.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.605617,0.643362,0.650314,0.508993,13.677419,10.677419,150.806452,8.709677,8.709677,5.129032,17661.580645,17644.580645,15354.903226,17105.387097,17985.870968,13810.806452
std,0.265190,0.256849,0.265586,0.306192,11.232059,9.474130,244.045818,9.103822,9.103822,4.951376,26406.623136,26470.523231,24237.338339,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.446578,0.504957,0.484027,0.232324,6.000000,4.000000,15.000000,2.000000,2.000000,1.500000,879.500000,852.500000,517.000000,802.000000,802.000000,410.000000
50%,0.630594,0.637417,0.717010,0.551845,11.000000,8.000000,49.000000,6.000000,6.000000,4.000000,4741.000000,4737.000000,2979.000000,3847.000000,4138.000000,1944.000000
75%,0.843416,0.869850,0.893239,0.770212,18.000000,14.000000,154.000000,13.500000,13.500000,7.000000,32332.000000,32218.000000,30411.500000,26305.000000,27036.500000,23520.500000
max,0.939784,0.948976,0.951051,0.919326,43.000000,36.000000,1048.000000,43.000000,43.000000,21.000000,100543.000000,100034.000000,91456.000000,102770.000000,105729.000000,92221.000000


In [29]:
# Median
df = ensemble_inference(models, test_df, median, threshold=0.4)
df.to_csv(f'{path}/ensemble_median.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.538917,0.571657,0.574137,0.463418,5.870968,5.258065,8.129032,8.709677,8.709677,5.129032,14391.806452,14530.451613,12065.064516,17105.387097,17985.870968,13810.806452
std,0.303437,0.294960,0.304586,0.338225,5.920259,5.409669,9.969092,9.103822,9.103822,4.951376,23191.244583,23455.686142,20697.480201,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.317495,0.385590,0.374568,0.123270,2.000000,2.000000,2.500000,2.000000,2.000000,1.500000,465.500000,463.000000,256.000000,802.000000,802.000000,410.000000
50%,0.509097,0.561654,0.563958,0.432812,5.000000,4.000000,5.000000,6.000000,6.000000,4.000000,2746.000000,2909.000000,2021.000000,3847.000000,4138.000000,1944.000000
75%,0.836340,0.861178,0.881641,0.770279,7.000000,5.500000,10.000000,13.500000,13.500000,7.000000,26027.000000,26167.000000,23096.000000,26305.000000,27036.500000,23520.500000
max,0.950051,0.955172,0.957395,0.937587,26.000000,23.000000,42.000000,43.000000,43.000000,21.000000,87741.000000,87904.000000,77850.000000,102770.000000,105729.000000,92221.000000


In [30]:
# Maximum Confidence
df = ensemble_inference(models, test_df, max_confidence, threshold=0.4)
df.to_csv(f'{path}ensemble_max_confidence.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.0000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.591759,0.629597,0.637825,0.504159,19.483871,17.580645,89.0000,8.709677,8.709677,5.129032,19367.322581,19508.000000,17485.838710,17105.387097,17985.870968,13810.806452
std,0.268760,0.261602,0.272585,0.292832,12.935406,13.180223,152.3986,9.103822,9.103822,4.951376,28103.705865,28352.972036,26499.924684,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.0000,1.000000,1.000000,0.000000,5.000000,2.000000,0.000000,44.000000,44.000000,0.000000
25%,0.371002,0.466939,0.458415,0.260921,9.000000,8.000000,11.0000,2.000000,2.000000,1.500000,1158.000000,1172.500000,745.500000,802.000000,802.000000,410.000000
50%,0.657639,0.691783,0.743555,0.578174,16.000000,12.000000,25.0000,6.000000,6.000000,4.000000,5192.000000,5246.000000,3982.000000,3847.000000,4138.000000,1944.000000
75%,0.827439,0.856369,0.866257,0.758419,29.000000,26.500000,82.0000,13.500000,13.500000,7.000000,35242.000000,35306.000000,33239.000000,26305.000000,27036.500000,23520.500000
max,0.924995,0.937072,0.938124,0.899788,53.000000,52.000000,633.0000,43.000000,43.000000,21.000000,106717.000000,107006.000000,99950.000000,102770.000000,105729.000000,92221.000000


In [31]:
# Majority Voting
majority_vote_df = ensemble_inference(models, test_df, majority_voting)
majority_vote_df.to_csv(f'{path}ensemble_majority_vote.csv', index=False)
majority_vote_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.554642,0.583924,0.590844,0.477403,6.193548,5.645161,8.354839,8.709677,8.709677,5.129032,14988.645161,15144.580645,12698.645161,17105.387097,17985.870968,13810.806452
std,0.294490,0.288909,0.293340,0.330313,6.420900,5.885283,10.731724,9.103822,9.103822,4.951376,24014.155578,24282.418701,21525.714756,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.367777,0.442381,0.404659,0.149619,2.000000,2.000000,2.000000,2.000000,2.000000,1.500000,507.500000,507.500000,317.500000,802.000000,802.000000,410.000000
50%,0.510321,0.567679,0.570280,0.454534,5.000000,4.000000,4.000000,6.000000,6.000000,4.000000,3098.000000,3272.000000,2246.000000,3847.000000,4138.000000,1944.000000
75%,0.840903,0.866662,0.884677,0.775159,7.500000,6.000000,9.000000,13.500000,13.500000,7.000000,27495.000000,27667.000000,25094.500000,26305.000000,27036.500000,23520.500000
max,0.947089,0.952529,0.955163,0.933574,30.000000,27.000000,43.000000,43.000000,43.000000,21.000000,91055.000000,91149.000000,81208.000000,102770.000000,105729.000000,92221.000000


In [32]:
# Dempter-Shafer
ds_df = ensemble_inference(models, test_df, dempster_shafer_inference, threshold=0.1)
ds_df.to_csv(f'{path}ensemble_dempster_shafer.csv', index=False)
ds_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.577296,0.603725,0.616029,0.494138,6.774194,5.967742,11.451613,8.709677,8.709677,5.129032,16001.161290,16154.677419,13782.000000,17105.387097,17985.870968,13810.806452
std,0.283266,0.279498,0.281408,0.319218,6.571198,5.805077,15.037373,9.103822,9.103822,4.951376,25349.161554,25590.406769,22925.056378,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.409390,0.514237,0.511225,0.181478,2.500000,2.500000,3.500000,2.000000,2.000000,1.500000,575.500000,582.500000,424.500000,802.000000,802.000000,410.000000
50%,0.551461,0.598369,0.601221,0.493097,5.000000,5.000000,6.000000,6.000000,6.000000,4.000000,3697.000000,3632.000000,2629.000000,3847.000000,4138.000000,1944.000000
75%,0.845099,0.874625,0.886767,0.776769,7.500000,6.500000,11.500000,13.500000,13.500000,7.000000,30352.000000,30570.000000,28244.500000,26305.000000,27036.500000,23520.500000
max,0.940381,0.946734,0.949310,0.925100,25.000000,26.000000,58.000000,43.000000,43.000000,21.000000,96632.000000,96629.000000,86888.000000,102770.000000,105729.000000,92221.000000


In [33]:
# CRF - Mean Field Approximation
df = ensemble_inference(models, test_df, crf_mean, threshold=0.4)
df.to_csv(f'{path}ensemble_crf_mean.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.605497,0.641875,0.651633,0.506557,13.741935,9.838710,162.161290,8.709677,8.709677,5.129032,17362.645161,17317.677419,14967.612903,17105.387097,17985.870968,13810.806452
std,0.265536,0.257429,0.263442,0.308732,12.521895,8.165361,263.428054,9.103822,9.103822,4.951376,26069.468031,26089.228239,23814.192373,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.460574,0.521829,0.512217,0.224473,4.500000,4.500000,15.500000,2.000000,2.000000,1.500000,834.500000,809.500000,477.000000,802.000000,802.000000,410.000000
50%,0.622315,0.642066,0.717188,0.550936,10.000000,7.000000,47.000000,6.000000,6.000000,4.000000,4672.000000,4601.000000,2860.000000,3847.000000,4138.000000,1944.000000
75%,0.842275,0.870411,0.895477,0.772928,18.500000,13.000000,163.500000,13.500000,13.500000,7.000000,31703.500000,31590.000000,29868.000000,26305.000000,27036.500000,23520.500000
max,0.942068,0.950584,0.952914,0.922705,50.000000,31.000000,1176.000000,43.000000,43.000000,21.000000,99290.000000,98733.000000,89919.000000,102770.000000,105729.000000,92221.000000


In [34]:
# CRF - Energy Minimization
df = ensemble_inference(models, test_df, crf_energy, threshold=0.4)
df.to_csv(f'{path}ensemble_crf_energy.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.605683,0.641933,0.652164,0.506371,13.580645,9.903226,160.741935,8.709677,8.709677,5.129032,17350.580645,17293.774194,14975.451613,17105.387097,17985.870968,13810.806452
std,0.265386,0.257282,0.263292,0.308636,12.510727,8.580423,260.579862,9.103822,9.103822,4.951376,26070.477501,26083.192592,23822.433151,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.465194,0.526412,0.525900,0.224549,4.500000,3.500000,15.500000,2.000000,2.000000,1.500000,831.500000,803.500000,476.500000,802.000000,802.000000,410.000000
50%,0.622171,0.643253,0.713107,0.550534,10.000000,7.000000,48.000000,6.000000,6.000000,4.000000,4647.000000,4595.000000,2864.000000,3847.000000,4138.000000,1944.000000
75%,0.842798,0.870454,0.895478,0.772731,18.000000,13.000000,161.000000,13.500000,13.500000,7.000000,31703.500000,31588.000000,29882.500000,26305.000000,27036.500000,23520.500000
max,0.942006,0.950542,0.952940,0.922537,50.000000,31.000000,1151.000000,43.000000,43.000000,21.000000,99268.000000,98665.000000,89945.000000,102770.000000,105729.000000,92221.000000


In [35]:
# Data
transforms = Transforms(seed=33)
test_dataset = EnsembleDataset(test_df.drop(columns = ['SubjectID']), transform=transforms.val_ensemble(), size = None, include_models=[True, False, False, True])
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle = False, sampler = test_sampler)
subjects = pd.read_csv('./data/TEST.csv')['SubjectID'].values

# Model
model = LogisticRegression(6, 3)
model.load_state_dict(torch.load(f'{path}LogRegCheckpoints/LogisticRegression_4.pt'))

#---------------------------------------------------------#

# Logistic Regression - No TTA
df = model_ensemble_inference(subjects, test_loader, model, spatial_size=6, threshold=0.9)
df.to_csv(f'{path}ensemble_logreg.csv', index=False)
df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.583115,0.628040,0.640094,0.477012,13.870968,12.612903,12.483871,8.709677,8.709677,5.129032,19690.322581,19310.193548,18150.483871,17105.387097,17985.870968,13810.806452
std,0.264822,0.267097,0.274947,0.276754,11.194469,11.692383,15.244389,9.103822,9.103822,4.951376,28819.670283,28435.020747,27167.190430,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,3.000000,3.000000,0.000000,44.000000,44.000000,0.000000
25%,0.392455,0.470563,0.473526,0.255282,7.500000,4.500000,4.000000,2.000000,2.000000,1.500000,1108.000000,1056.500000,755.000000,802.000000,802.000000,410.000000
50%,0.649034,0.673607,0.717494,0.511213,10.000000,9.000000,9.000000,6.000000,6.000000,4.000000,5276.000000,5139.000000,4812.000000,3847.000000,4138.000000,1944.000000
75%,0.810600,0.858849,0.878384,0.698885,16.000000,15.000000,14.000000,13.500000,13.500000,7.000000,36119.500000,35745.000000,34043.000000,26305.000000,27036.500000,23520.500000
max,0.916859,0.928759,0.934520,0.887298,52.000000,55.000000,81.000000,43.000000,43.000000,21.000000,108875.000000,107411.000000,102438.000000,102770.000000,105729.000000,92221.000000
